## Week 3:

New Topics:

 * Exploratory Data Analysis
  

Coding Task:

This week, you will build off of your work from last week.
1. First, (if you haven't done so already), create a new column indicating the SHADAC classification for each Tennessee county. You can use the following code to accomplish this:
```
physicians.loc[physicians['residents_per_pcp'] < 1500, 'shadac_category'] = 'adequate'
physicians.loc[(physicians['residents_per_pcp'] >= 1500) & 
          (physicians['residents_per_pcp'] < 3500), 'shadac_category'] = 'moderately inadequate'
physicians.loc[(physicians['residents_per_pcp'] >= 3500), 'shadac_category'] = 'low inadequate'
```

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read in data files
physicians = pd.read_csv('../data/primary_care_physicians.csv')
physicians.head()
population = pd.read_csv('../data/population_by_county.csv')

In [8]:
# Correct column variables and merge data files; create new coluumn for residents per pcp
population['county'].str.split(' County', expand = True)
population['county']=population['county'].str.split(' County', expand = True)[0]
population = population.loc[population['state'] == 'TENNESSEE']
physicians = pd.merge(left = physicians,
         right = population[['county','population', 'urban']])
physicians['residents_per_pcp'] = physicians['population'] / physicians['primary_care_physicians']

In [9]:
# Create categories for level of care
physicians.loc[physicians['residents_per_pcp'] < 1500, 'shadac_category'] = 'adequate'
physicians.loc[(physicians['residents_per_pcp'] >= 1500) & 
       (physicians['residents_per_pcp'] < 3500), 'shadac_category'] = 'moderately inadequate'
physicians.loc[(physicians['residents_per_pcp'] >= 3500), 'shadac_category'] = 'low inadequate'

2. Use this new column to investigate if there is any relationship between a county's status as urban or rural and its SHADAC classification. Create a plot showing what you find.

In [ ]:
physicians[''].value_counts().plot(kind = 'bar');

3. Merge the unemployment data (contained in tn_unemployment.csv) into the physicians DataFrame.

In [18]:
unemployment = pd.read_csv('../data/tn_unemployment.csv')
#unemployment.head()

In [19]:
unemployment['Name'].str.split(' County', expand = True)
unemployment['Name'] = unemployment['Name'].str.split(' County', expand = True)[0]
#unemployment.head()

In [20]:
physicians = pd.merge(left = physicians,
         right = unemployment.rename(columns = {'Name': 'county'}))
physicians.head()

,FIPS,state,county,primary_care_physicians,population,urban,residents_per_pcp,shadac_category,laus_code,State,County,Period,LF,Employed,Unemployed,unemployment_rate
0,1009,Alabama,Blount,12.0,129927,Urban,10827.250000,low inadequate,CN4700900000000,47,9,Mar-21,64098,61119,2979,4.6
1,47009,Tennessee,Blount,90.0,129927,Urban,1443.633333,adequate,CN4700900000000,47,9,Mar-21,64098,61119,2979,4.6
2,1027,Alabama,Clay,5.0,7654,Rural,1530.800000,moderately inadequate,CN4702700000000,47,27,Mar-21,2743,2557,186,6.8
3,5021,Arkansas,Clay,1.0,7654,Rural,7654.000000,low inadequate,CN4702700000000,47,27,Mar-21,2743,2557,186,6.8
4,12019,Florida,Clay,113.0,7654,Rural,67.734513,adequate,CN4702700000000,47,27,Mar-21,2743,2557,186,6.8


4. How do unemployment rates compare for urban counties versus rural counties?

5. Create a new column, pcp_per_100k which contains the number of primary care physicians per 100,000 residents. Investigate the relationship between this new measure and the unemployment rate per county. What do you find?

In [21]:
physicians['pcp_per_100k'] = physicians['population'] / physicians['primary_care_physicians']